python3 -m pip install -U git+https://github.com/adefossez/demucs#egg=demucs

In [16]:
import demucs.api
import soundfile as sf
import torchaudio
import torch
import scipy.io as sio
import tensorflow as tf
import scipy
import numpy as np
import csv

# import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile

# import kagglehub
# import tensorflow_hub as hub

In [2]:
# Download latest version
# path = kagglehub.model_download("google/yamnet/tensorFlow2/yamnet")
path = 'yamnet/tensorFlow2/yamnet/1'

print("Path to model files:", path)

Path to model files: yamnet/tensorFlow2/yamnet/1


In [3]:
model = tf.saved_model.load(
    path, tags=None, options=None
)

In [4]:
# Initialize with default parameters:
separator = demucs.api.Separator()

In [5]:
# import librosa
# y, sr = librosa.load('/Users/andrewchang/NYU_research/MusicSpeech-STM/data/musicCorp/fma_large/000/000002.mp3')
# y, sr = sf.read('/Users/andrewchang/NYU_research/MusicSpeech-STM/data/musicCorp/fma_large/000/000002.mp3')
y, sr = torchaudio.load('/Users/andrewchang/NYU_research/MusicSpeech-STM/data/musicCorp/fma_large/000/000002.mp3')


In [6]:
y.shape

torch.Size([2, 1321344])

In [7]:
sr

44100

In [8]:
y_tensor = y

In [9]:
# import numpy as np
# import torch
# # y_2 = np.stack((y, y))
# y_tensor = torch.tensor(y)

# if y_tensor.shape[0] > y_tensor.shape[1]:
#     y_tensor = y_tensor.T

# Duplicate and stack the tensor
if y_tensor.shape[0]==1:
    y_tensor = torch.stack((y_tensor, y_tensor), dim=0)

y_tensor.shape

torch.Size([2, 1321344])

In [10]:
# this function by default will make sure the sampling rate is at 44.1 kHz
origin, separated = separator.separate_tensor(wav=y_tensor, sr=sr)


In [11]:
separated

{'drums': tensor([[0.0003, 0.0003, 0.0003,  ..., 0.0013, 0.0007, 0.0009],
         [0.0004, 0.0003, 0.0003,  ..., 0.0017, 0.0008, 0.0019]]),
 'bass': tensor([[0.0004, 0.0004, 0.0004,  ..., 0.0184, 0.0181, 0.0180],
         [0.0004, 0.0004, 0.0005,  ..., 0.0211, 0.0207, 0.0203]]),
 'other': tensor([[-2.1604e-04, -2.4037e-05, -5.9582e-05,  ..., -6.5464e-03,
          -1.1816e-02, -1.4527e-02],
         [ 3.3266e-05,  2.1111e-04,  2.2579e-04,  ..., -1.0578e-03,
          -1.1904e-02, -1.3923e-02]]),
 'vocals': tensor([[0.0004, 0.0004, 0.0004,  ..., 0.0921, 0.0774, 0.0425],
         [0.0003, 0.0003, 0.0004,  ..., 0.1129, 0.1449, 0.1349]])}

In [12]:
separated['vocals'].shape

torch.Size([2, 1321344])

In [13]:
separated['vocals'].mean(axis=0)

tensor([0.0003, 0.0003, 0.0004,  ..., 0.1025, 0.1112, 0.0887])

In [17]:
# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
  """Returns list of class names corresponding to score vector."""
  class_names = []
  with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      class_names.append(row['display_name'])

  return class_names

class_map_path = model.class_map_path().numpy()
class_names = class_names_from_csv(class_map_path)

# verify and convert a loaded audio is on the proper sample_rate (16K), otherwise it would affect the model's results.

def ensure_sample_rate(original_sample_rate, waveform,
                       desired_sample_rate=16000):
  """Resample waveform if required."""
  if original_sample_rate != desired_sample_rate:
    desired_length = int(round(float(len(waveform)) /
                               original_sample_rate * desired_sample_rate))
    waveform = scipy.signal.resample(waveform, desired_length)
  return desired_sample_rate, waveform

In [18]:
_, waveform = ensure_sample_rate(sr, np.array(separated['vocals'].mean(axis=0)))

In [19]:
scores, embeddings, spectrogram = model(np.array(separated['vocals'].mean(axis=0)))

In [20]:
scores

<tf.Tensor: shape=(172, 521), dtype=float32, numpy=
array([[5.0781357e-01, 1.9586347e-03, 4.9304729e-04, ..., 1.3524513e-03,
        9.8279375e-04, 1.2134526e-08],
       [1.8021800e-01, 7.3524085e-03, 1.8393683e-03, ..., 2.4640195e-03,
        6.5983087e-03, 3.2770197e-07],
       [4.2811926e-02, 7.6749391e-04, 8.6811907e-04, ..., 1.5474551e-03,
        2.0208836e-03, 4.6398277e-08],
       ...,
       [6.3549966e-02, 8.9087925e-04, 8.3256915e-04, ..., 3.7630391e-03,
        2.7444386e-03, 3.5474150e-06],
       [4.5188186e-03, 1.2748324e-03, 3.7926813e-05, ..., 4.9572048e-04,
        3.5652544e-04, 1.5283793e-07],
       [7.8315310e-02, 6.5313084e-03, 2.6652061e-03, ..., 1.6982110e-02,
        1.1271269e-02, 1.7485968e-04]], dtype=float32)>

In [21]:
scores_np = scores.numpy()
spectrogram_np = spectrogram.numpy()
infered_class = class_names[scores_np.mean(axis=0).argmax()]
print(f'The main sound is: {infered_class}')

The main sound is: Speech


In [23]:
scores_np

array([[4.5364147e-01, 4.5595123e-03, 1.1929839e-03, ..., 2.7698800e-03,
        1.2314862e-03, 9.1753968e-08],
       [1.6486099e-01, 1.3112247e-02, 1.0731223e-03, ..., 3.2815991e-03,
        3.9036016e-03, 6.9473472e-08],
       [3.8418364e-02, 1.4409499e-03, 1.1317354e-03, ..., 5.7045547e-03,
        4.5717005e-03, 7.2389389e-07],
       ...,
       [5.4133873e-02, 6.0582132e-04, 3.6296496e-04, ..., 1.7845613e-03,
        1.0789689e-03, 9.4569430e-07],
       [6.6219610e-03, 3.1086968e-03, 4.8757669e-05, ..., 4.1922869e-04,
        1.9914478e-04, 1.6626602e-08],
       [8.1586674e-02, 3.1353948e-03, 3.2573056e-03, ..., 2.8697215e-02,
        9.8005384e-03, 1.2756771e-04]], dtype=float32)